In [1]:
from llm_agent.cell_inspectors import ExcelVariableAgent
from llm_agent.programmer import ProgrammerAgent
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
spreadsheet_name = 'simple_beam'
path = f"examples/simple_beam/{spreadsheet_name}.xlsx"

variable_agent = ExcelVariableAgent(
                 spread_sheet_path =path,
                 system_prompt = None,
                 openai_model = "gpt-4.1-mini",
                 trace_with_langfuse = True)

INFO:db.database:Connecting to database: excel_agents
INFO:db.database:Successfully connected to MongoDB
INFO:db.excel_manager:Deleted spreadsheet 'simple_beam' (ID: 683cd7e73f407fa7f60bd958) with 55 cells
INFO:db.excel_manager:Parsing spreadsheet: examples/simple_beam/simple_beam.xlsx
INFO:parsers.excel_parser:Parsing Excel file: examples/simple_beam/simple_beam.xlsx
INFO:parsers.excel_parser:Building alias mapping from defined names
INFO:parsers.excel_parser:Found 3 cell aliases across 1 sheets
INFO:parsers.excel_parser:Processing sheet: calcs
INFO:parsers.excel_parser:Sheet calcs dimensions: 27 rows x 12 columns
INFO:parsers.excel_parser:Expanded options: [{'cell_ref': 'J24', 'sheet_name': 'calcs', 'workbook_name': 'simple_beam.xlsx'}, {'cell_ref': 'J25', 'sheet_name': 'calcs', 'workbook_name': 'simple_beam.xlsx'}, {'cell_ref': 'J26', 'sheet_name': 'calcs', 'workbook_name': 'simple_beam.xlsx'}, {'cell_ref': 'J27', 'sheet_name': 'calcs', 'workbook_name': 'simple_beam.xlsx'}]
INFO:par

In [3]:
variable_agent.project_dir

'/Users/omid/Desktop/Projects/excel_agent/examples/simple_beam'

In [4]:
await variable_agent.orchestrate_variable_extraction()

processing layer 0 with 18 cells
processing layer 1 with 5 cells
processing layer 2 with 2 cells
processing layer 3 with 1 cells


In [5]:
for k,v in variable_agent.variable_db.items():
    sheet,cell = k
    print(sheet,cell)
    var_name = v['variable_name']
    variable_desr = v['variable_desr']
    python_code = v['python_code']
    print(f"var name: {var_name}")
    print(f"var desc: {variable_desr}")
    print(f"python code:\n`{python_code}`")
    print("-"*100)

calcs D3
var name: beam_length_ft
var desc: length of the beam in feet
python code:
`beam_length_ft = 20`
----------------------------------------------------------------------------------------------------
calcs D4
var name: P_kip
var desc: mid span point load in kips
python code:
`P_kip = 15`
----------------------------------------------------------------------------------------------------
calcs D5
var name: area_load_q
var desc: area load in psf
python code:
`area_load_q = 30`
----------------------------------------------------------------------------------------------------
calcs D6
var name: trib_width_ft
var desc: tributary width in feet
python code:
`trib_width_ft = 20`
----------------------------------------------------------------------------------------------------
calcs D7
var name: beam_section
var desc: The section type of the beam, which can be one of the finite set values: 'alpha', 'bravo', 'charlie', or 'delta'.
python code:
`beam_section = 'delta'  # Example value,

In [6]:
# layer = variable_agent.layers[0]
# _  = await variable_agent.process_layer(layer)

Testing context methods:

In [7]:
context = variable_agent.build_cell_processing_prompt(cell_id="D7",sheetname="calcs")
print(context)

Cell D7 in sheet 'calcs' can have a finite set of input values defined as follows:
| Value |
| alpha |
| bravo |
| charlie |
| delta |

The cell tile data for cell D7 in sheet 'calcs' is as follows:
|   | A | B | C | D | E | F | G | H | I |
|---|---|---|---|---|---|---|---|---|---|
| 2 |  |  |  |  |  |  |  |  |  |
| 3 |  |  | L [ft] | 20 |  | length of the beam |  |  |  |
| 4 |  |  | P [kip] | 15 |  | mid span point load |  |  |  |
| 5 |  |  | q [psf] | 30 |  | area load |  |  |  |
| 6 |  |  | Trib [ft] | 20 |  | trib width |  |  |  |
| 7 |  |  | Section | delta |  |  |  |  |  |
| 8 |  |  |  |  |  |  |  |  |  |
| 9 |  |  |  |  |  |  |  | Moment capacity | Mcap |
| 10 |  |  |  |  |  |  |  | shear capacity | Vcap |
| 11 |  |  | This spreadsheet is used to check 
a simple beam |  |  |  |  |  |  |
| 12 |  |  |  |  |  |  |  |  |  |


In [8]:
context = variable_agent.build_cell_processing_prompt(cell_id="D3",sheetname="calcs")
print(context)

Cell D3 in sheet 'calcs' contains the contast value: 20. 
The cell tile data for cell D3 in sheet 'calcs' is as follows:
|   | A | B | C | D | E | F | G | H | I |
|---|---|---|---|---|---|---|---|---|---|
| 1 |  |  |  |  |  |  |  |  |  |
| 2 |  |  |  |  |  |  |  |  |  |
| 3 |  |  | L [ft] | 20 |  | length of the beam |  |  |  |
| 4 |  |  | P [kip] | 15 |  | mid span point load |  |  |  |
| 5 |  |  | q [psf] | 30 |  | area load |  |  |  |
| 6 |  |  | Trib [ft] | 20 |  | trib width |  |  |  |
| 7 |  |  | Section | delta |  |  |  |  |  |
| 8 |  |  |  |  |  |  |  |  |  |


In [9]:
context = variable_agent.build_cell_processing_prompt(cell_id="E19",sheetname="calcs")
print(context)

Cell E19 in sheet 'calcs' contains the formula: =P*D3/4+(D5*D6/1000)*D3^2/8. Dependent cell information is as follows:
| Precedent Cell ID | Sheet Name | Variable Name | Variable Description | Python Code |
| D3 | calcs | beam_length_ft | length of the beam in feet | beam_length_ft = 20 |
| D5 | calcs | area_load_q | area load in psf | area_load_q = 30 |
| D6 | calcs | trib_width_ft | tributary width in feet | trib_width_ft = 20 |
| D4 | calcs | P_kip | mid span point load in kips | P_kip = 15 |

The cell tile data for cell E19 in sheet 'calcs' is as follows:
|   | A | B | C | D | E | F | G | H | I | J |
|---|---|---|---|---|---|---|---|---|---|---|
| 14 |  |  |  |  |  |  |  |  |  |  |
| 15 |  |  | Max P | 100 |  |  |  |  |  |  |
| 16 |  |  | Check max P | Load below limit |  |  |  |  |  |  |
| 17 |  |  |  |  |  |  |  |  |  |  |
| 18 |  |  |  |  |  |  |  |  |  |  |
| 19 |  |  | Moment | M | 105 | kip.ft |  |  |  |  |
| 20 |  |  | Shear | V | 156 | kip.ft |  |  |  |  |
| 21 |  |  |  |  

In [10]:
programmer = ProgrammerAgent(
    spread_sheet_path=path,
    system_prompt=None,
    openai_model="gpt-4.1",
    trace_with_langfuse=True
)

In [11]:
code_context = programmer.build_coding_context()
print(code_context)


#beam_length_ft:length of the beam in feet
beam_length_ft = 20
#P_kip:mid span point load in kips
P_kip = 15
#area_load_q:area load in psf
area_load_q = 30
#trib_width_ft:tributary width in feet
trib_width_ft = 20
#beam_section:The section type of the beam, which can be one of the finite set values: 'alpha', 'bravo', 'charlie', or 'delta'.
beam_section = 'delta'  # Example value, can be 'alpha', 'bravo', 'charlie', or 'delta'
#alpha:Constant value alpha from section database in sheet calcs
alpha = 20
#bravo:Constant value named bravo from the section database in the calcs sheet.
'bravo'
#section_name_charlie:Section name corresponding to the row with alpha, bravo, charlie, delta in the section database
'charlie'
#section_name_delta:Section name 'delta' from the section database in the calcs sheet
'delta'
#alpha_M_kip_ft:Moment M in kip.ft for section alpha from the section database
alpha_M_kip_ft = 20
#bravo_M_kip_ft:Moment M in kip.ft for section named bravo from the section database

In [12]:
messages = await programmer.initialize_coding_agent()

INFO:root:Created /Users/omid/Desktop/Projects/excel_agent/examples/simple_beam/beam_design
INFO:root:Created /Users/omid/Desktop/Projects/excel_agent/examples/simple_beam/beam_design/example.py
INFO:root:Created /Users/omid/Desktop/Projects/excel_agent/examples/simple_beam/beam_design/calculations.py
INFO:root:Created /Users/omid/Desktop/Projects/excel_agent/examples/simple_beam/beam_design/readme.md
